# Sample static LCIA results


In [2]:
import bw2analyzer as bwa
import bw2calc as bc
import bw2data as bd
import bw2io as bi
#import ipywidgets as widgets
import pandas as pd
from bw2data.query import Filter, Query
from IPython.display import display

In [3]:
from project_details import EI_DB_NAME, PROJECT_NAME

In [4]:
bd.projects.set_current(PROJECT_NAME)
bd.databases

Databases dictionary with 3 object(s):
	asphalt
	biosphere3
	ecoinvent-391-cutoff

In [5]:
# Is the background database name the same as the one we wrote in `propject_details.py`?
assert EI_DB_NAME in bd.databases

In [6]:
FG_DB_NAME = "asphalt"

In [7]:
db_asphalt = bd.Database(FG_DB_NAME)

In [8]:
pavement_complete_a = db_asphalt.get("DZOAB, A")

In [9]:
bwa.print_recursive_supply_chain(pavement_complete_a, max_level=1)

1: 'DZOAB, A' (kilogram, NL, None)
  1: 'A1, pavement, materials, A' (kilogram, NL, None)
  1: 'A2, pavement, transport to plant, A' (kilogram, NL, None)
  1: 'A3, pavement, production, A' (kilogram, NL, None)
  1: 'A4, pavement, transport to site' (kilogram, NL, None)
  1: 'A5, pavement, construction' (kilogram, NL, None)
  1: 'C1, pavement, demolition' (kilogram, NL, None)
  1: 'C2, pavement, transport to processing' (kilogram, NL, None)
  1: 'C3, pavement, processing' (kilogram, NL, None)


In [10]:
bwa.print_recursive_supply_chain(pavement_complete_a, max_level=2)

1: 'DZOAB, A' (kilogram, NL, None)
  1: 'A1, pavement, materials, A' (kilogram, NL, None)
    0.3: 'asphalt granulate, free of burden' (kilogram, NL, None)
    0.0412: 'bitumen adhesive compound production, hot' (kilogram, RER, None)
    0.586: 'crushed stone, from quarry in Europe, excluding transport to the Netherlands' (kilogram, NL, None)
    0.0094: 'crushed stone, from quarry in Europe, excluding transport to the Netherlands' (kilogram, NL, None)
    0.0342: 'gravel production, crushed' (kilogram, RoW, None)
    0.0021: 'cellulose fibre production' (kilogram, RoW, None)
    0.027: 'medium filler' (kilogram, NL, None)
  1: 'A2, pavement, transport to plant, A' (kilogram, NL, None)
    0.0103: 'market for transport, freight, lorry, unspecified' (ton kilometer, RER, None)
    0.547: 'market for transport, freight, sea, ferry' (ton kilometer, GLO, None)
    0.0311: 'market for transport, freight, inland waterways, barge' (ton kilometer, RER, None)
    0.0147: 'market for transport, f

## Find the activities in the db providing "pavement, complete" as ref prod

In [11]:
asphalt_db_data = db_asphalt.load()

In [12]:
query = Query()
filter_refprod = Filter("reference product", "is", "pavement, complete")
query.add(filter_refprod)

In [13]:
res = query(asphalt_db_data)

In [14]:
for res_key, res_act in res.items():
    print(bd.get_activity(res_key))

'DZOAB, B, PVI' (kilogram, NL, None)
'DZOAB, A' (kilogram, NL, None)
'DZOAB, A, PVI' (kilogram, NL, None)
'DZOAB, B' (kilogram, NL, None)


## LCIA

### Functional units

In [15]:
# Use the found activities providing "pavement, complete" as functional units for a multi lca
functional_units = [{bd.get_activity(res_a): 1} for res_a, _ in res.items()]
functional_units

[{'DZOAB, B, PVI' (kilogram, NL, None): 1},
 {'DZOAB, A' (kilogram, NL, None): 1},
 {'DZOAB, A, PVI' (kilogram, NL, None): 1},
 {'DZOAB, B' (kilogram, NL, None): 1}]

In [16]:
# We sort the obtained functional units by code + ref prod
functional_units = sorted(
    functional_units,
    key=lambda fu: [
        list(fu.keys())[0]["code"],
        list(fu.keys())[0]["reference product"],
    ],
)
functional_units

[{'DZOAB, A' (kilogram, NL, None): 1},
 {'DZOAB, A, PVI' (kilogram, NL, None): 1},
 {'DZOAB, B' (kilogram, NL, None): 1},
 {'DZOAB, B, PVI' (kilogram, NL, None): 1}]

### LCIA Methods

In [17]:
bd.methods

Methods dictionary with 762 objects, including:
	('CML v4.8 2016', 'acidification', 'acidification (incl. fate, average Europe total, A&B)')
	('CML v4.8 2016', 'climate change', 'global warming potential (GWP100)')
	('CML v4.8 2016', 'ecotoxicity: freshwater', 'freshwater aquatic ecotoxicity (FAETP inf)')
	('CML v4.8 2016', 'ecotoxicity: marine', 'marine aquatic ecotoxicity (MAETP inf)')
	('CML v4.8 2016', 'ecotoxicity: terrestrial', 'terrestrial ecotoxicity (TETP inf)')
	('CML v4.8 2016', 'energy resources: non-renewable', 'abiotic depletion potential (ADP): fossil fuels')
	('CML v4.8 2016', 'eutrophication', 'eutrophication (fate not incl.)')
	('CML v4.8 2016', 'human toxicity', 'human toxicity (HTP inf)')
	('CML v4.8 2016', 'material resources: metals/minerals', 'abiotic depletion potential (ADP): elements (ultimate reserves)')
	('CML v4.8 2016', 'ozone depletion', 'ozone layer depletion (ODP steady state)')
Use `list(this object)` to get the complete list.

In [24]:
cml_methods = [m for m in bd.methods if m[0] == "CML v4.8 2016"]
cml_methods

[('CML v4.8 2016',
  'acidification',
  'acidification (incl. fate, average Europe total, A&B)'),
 ('CML v4.8 2016', 'climate change', 'global warming potential (GWP100)'),
 ('CML v4.8 2016',
  'ecotoxicity: freshwater',
  'freshwater aquatic ecotoxicity (FAETP inf)'),
 ('CML v4.8 2016',
  'ecotoxicity: marine',
  'marine aquatic ecotoxicity (MAETP inf)'),
 ('CML v4.8 2016',
  'ecotoxicity: terrestrial',
  'terrestrial ecotoxicity (TETP inf)'),
 ('CML v4.8 2016',
  'energy resources: non-renewable',
  'abiotic depletion potential (ADP): fossil fuels'),
 ('CML v4.8 2016', 'eutrophication', 'eutrophication (fate not incl.)'),
 ('CML v4.8 2016', 'human toxicity', 'human toxicity (HTP inf)'),
 ('CML v4.8 2016',
  'material resources: metals/minerals',
  'abiotic depletion potential (ADP): elements (ultimate reserves)'),
 ('CML v4.8 2016',
  'ozone depletion',
  'ozone layer depletion (ODP steady state)'),
 ('CML v4.8 2016',
  'photochemical oxidant formation',
  'photochemical oxidation (h

In [25]:
bd.calculation_setups["asphalt_pavement_complete"] = {
    "inv": functional_units,
    "ia": cml_methods,
}

In [26]:
# `mlca.results`, is a NumPy array of LCA scores,
# with rows of functional units and
# columns of LCIA methods. Ordering is the same as in the `calculation_setup`.
mlca = bc.MultiLCA("asphalt_pavement_complete")

In [27]:
def format_results(mlca):
    formatted_results = []
    for i, scores in enumerate(mlca.results):  # the results for a fu
        for j, method_key in enumerate(mlca.methods):  # the result for each method
            demand = list(mlca.func_units[i].values())[0]
            activity = list(mlca.func_units[i].keys())[0]
            method = bd.Method(method_key)
            a_result = {
                "activity": repr(activity),
                "demand": demand,
                "method": method_key[0],
                "category": method_key[1],
                "subcategory": method_key[2],
                "score": scores[j],
                "unit": method.metadata["unit"],
            }
            formatted_results.append(a_result)
    return formatted_results

In [28]:
df_results = pd.DataFrame(format_results(mlca))
df_results

,activity,demand,method,category,subcategory,score,unit
0,"'DZOAB, A' (kilogram, NL, None)",1,CML v4.8 2016,acidification,"acidification (incl. fate, average Europe tota...",1.840688e-03,kg SO2-Eq
1,"'DZOAB, A' (kilogram, NL, None)",1,CML v4.8 2016,climate change,global warming potential (GWP100),1.600993e-01,kg CO2-Eq
2,"'DZOAB, A' (kilogram, NL, None)",1,CML v4.8 2016,ecotoxicity: freshwater,freshwater aquatic ecotoxicity (FAETP inf),3.178744e-02,"kg 1,4-DCB-Eq"
3,"'DZOAB, A' (kilogram, NL, None)",1,CML v4.8 2016,ecotoxicity: marine,marine aquatic ecotoxicity (MAETP inf),6.660882e+01,"kg 1,4-DCB-Eq"
4,"'DZOAB, A' (kilogram, NL, None)",1,CML v4.8 2016,ecotoxicity: terrestrial,terrestrial ecotoxicity (TETP inf),8.538967e-04,"kg 1,4-DCB-Eq"
5,"'DZOAB, A' (kilogram, NL, None)",1,CML v4.8 2016,energy resources: non-renewable,abiotic depletion potential (ADP): fossil fuels,3.633902e+00,megajoule
6,"'DZOAB, A' (kilogram, NL, None)",1,CML v4.8 2016,eutrophication,eutrophication (fate not incl.),2.463223e-04,kg PO4-Eq
7,"'DZOAB, A' (kilogram, NL, None)",1,CML v4.8 2016,human toxicity,human toxicity (HTP inf),5.633777e+00,"kg 1,4-DCB-Eq"
8,"'DZOAB, A' (kilogram, NL, None)",1,CML v4.8 2016,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,3.505378e-07,kg Sb-Eq
9,"'DZOAB, A' (kilogram, NL, None)",1,CML v4.8 2016,ozone depletion,ozone layer depletion (ODP steady state),4.802367e-09,kg CFC-11-Eq


In [21]:
df_results.to_excel("asphalt_lcia_results.xlsx")